# Initialize database files

In [13]:
%cd twitteranalysis
import environment
import sqlite3
import os
from Loggers.Helpers import delete_files


g = environment.sqlite_file_connection_string_generator()
query = """CREATE TABLE `word_map_deux` (
`tweet_id` int(20) DEFAULT NULL,
  `user_id` int(20) DEFAULT NULL,
  `word` varchar(200) DEFAULT NULL,
  `sentence_index` int(11) DEFAULT NULL,
  `word_index` int(11) DEFAULT NULL)"""

# Make master db and tables
import sqlite3

# make individual tables
def initialize_working_tables():
    for i in range(0, environment.MAX_DB_FILES +1):
        try:
            f = '%s/wordmapping%s.db' % (environment.DB_FOLDER, i)
            conn = sqlite3.connect(f)
            conn.execute(query)
            conn.commit()
            conn.close()
        except:
            pass

def initialize_master_db():
    try:
        conn = sqlite3.connect(environment.MASTER_DB)
        conn.execute(query)
        conn.commit()
        conn.close()
    except:
        pass

def delete_master_db():
    os.remove(environment.MASTER_DB)
    
def delete_working_files():
    delete_files(environment.DB_FOLDER)

(bookmark:twitteranalysis) -> /Users/adam/Dropbox/PainNarrativesLab/TwitterDataAnalysis
/Users/adam/Dropbox/PainNarrativesLab/TwitterDataAnalysis


In [16]:
delete_master_db()
initialize_master_db()

In [17]:
delete_working_files()
initialize_working_tables()

removing wordmapping3.db
removing wordmapping7.db
removing wordmapping6.db
removing wordmapping2.db
removing wordmapping9.db
removing wordmapping8.db
removing wordmapping5.db
removing wordmapping10.db
removing wordmapping1.db
removing wordmapping0.db
removing wordmapping4.db


# Run the user parser

In [1]:
%cd twitteranalysis
%run -i Executables/process_user_descriptions_into_words2.py

(bookmark:twitteranalysis) -> /Users/adam/Dropbox/PainNarrativesLab/TwitterDataAnalysis
/Users/adam/Dropbox/PainNarrativesLab/TwitterDataAnalysis
Starting run


RuntimeError: This event loop is already running

connection ready
Starting processing 


Exception in callback Client.send_flush_command(<Future finished result=4352>)
handle: <Handle Client.send_flush_command(<Future finished result=4352>)>
Traceback (most recent call last):
  File "/Users/adam/.pyenv/versions/3.6.4/lib/python3.6/asyncio/events.py", line 145, in _run
    self._callback(*self._args)
  File "/Users/adam/Dropbox/PainNarrativesLab/TwitterDataAnalysis/Servers/ClientSide.py", line 89, in send_flush_command
    for _ in range( 0, repeat ):
TypeError: '_asyncio.Future' object cannot be interpreted as an integer


4352 users processed
Processing complete. Processed 4352 users
requesting queue flush


In [ ]:
class AsyncBatcher(object):
    __slots__ = ['batch', 'batch_size', 'save', 'flush']
    def __init__(self, batch_size):
        self.batch_size = batch_size
        self.batch = []
    
    

In [ ]:
%cd twitteranalysis
from Executables.process_user_descriptions_into_words2 import main
main()


In [ ]:
%cd twitteranalysis
from Executables.process_user_descriptions_into_words2 import main
main()


# Clear queue

In [12]:
%cd twitteranalysis

from Servers.ClientSide import Client
c = Client()

# Each of the listening request handlers needs its queue flushed
c.send_flush_command()

#j = c.send_shutdown_command()

(bookmark:twitteranalysis) -> /Users/adam/Dropbox/PainNarrativesLab/TwitterDataAnalysis
/Users/adam/Dropbox/PainNarrativesLab/TwitterDataAnalysis
requesting queue flush


In [ ]:
j

# Merge the various sqlite files

In [18]:

%cd twitteranalysis
import environment
import sqlite3

mainDB = environment.MASTER_DB
columnNames = "word_index, sentence_index, word, user_id, tweet_id"
    
otherDbNames = [ 'wordmapping%s.db' % (i) 
            for i in range( 0, environment.MAX_DB_FILES +1)]

otherDBs = [ '%s/%s' % (environment.DB_FOLDER, name) 
            for name in otherDbNames]
    
for db in otherDbNames:
    conn = sqlite3.connect( mainDB )  # Connect to the main database
    curs = conn.cursor()  # Connect a cursor
    
    dbPath = '%s/%s' % (environment.DB_FOLDER, db)
    curs.execute( "ATTACH DATABASE ? as ? ;", (dbPath, 'db') )

    dbNameTableName = "db.word_map_deux"
    tableName = 'word_map_deux'
    curs.execute( "INSERT INTO %s (%s) SELECT %s FROM %s;" % (tableName, columnNames, columnNames, dbNameTableName) )
    conn.commit()
    curs.close()
    conn.close()


(bookmark:twitteranalysis) -> /Users/adam/Dropbox/PainNarrativesLab/TwitterDataAnalysis
/Users/adam/Dropbox/PainNarrativesLab/TwitterDataAnalysis


In [ ]:
otherDbNames

# Check data

In [19]:
%cd twitteranalysis
import environment
import sqlite3

(bookmark:twitteranalysis) -> /Users/adam/Dropbox/PainNarrativesLab/TwitterDataAnalysis
/Users/adam/Dropbox/PainNarrativesLab/TwitterDataAnalysis


In [20]:
uids = []
rows = []
print("Unique user ids; rows")
for db in otherDbNames:
    dbPath = '%s/%s' % (environment.DB_FOLDER, db)
    conn = sqlite3.connect( dbPath )  # Connect to the main database
    curs = conn.cursor()  # Connect a cursor
    r1 = conn.execute("select count( distinct user_id) from word_map_deux")
    v =  r1.fetchone()
    uids.append(v[0])
    r2 = conn.execute("select count( word) from word_map_deux")
    v2 =  r2.fetchone()
    rows.append(v2[0])

    print("%s : %s; %s " % (db, v[0], v2[0]))
    conn.close()


Unique user ids; rows
wordmapping0.db : 0; 0 
wordmapping1.db : 599; 9999 
wordmapping2.db : 544; 8997 
wordmapping3.db : 481; 8008 
wordmapping4.db : 493; 8008 
wordmapping5.db : 476; 8003 
wordmapping6.db : 435; 7007 
wordmapping7.db : 400; 6500 
wordmapping8.db : 280; 4489 
wordmapping9.db : 184; 3003 
wordmapping10.db : 167; 2493 


In [21]:
sum(uids)

4059

In [22]:
sum(rows)

66507

In [23]:
conn = sqlite3.connect(environment.MASTER_DB)
r1 = conn.execute("select count( distinct user_id) from word_map_deux")
print("%s Unique user ids" % r1.fetchone())
r2 = conn.execute("select count( word) from word_map_deux")
print("%s rows in master.db" % r2.fetchone())
conn.close()

3923 Unique user ids
66507 rows in master.db


NB: 
    - numberUsers processed = 4352
    - users w empty descriptions = 332
    - users non-english = 129
    - expected users: 3891

5/15 11.12
    - 4059 unique ids based on files
    - 3923 unique ids based on master
    - 66507 rows in master and based on files

5/15 10.39
    - 3653 unique ids based on files
    - 3529 unique ids based on master
    - 60060 rows in master and based on files
    

5/14 9.00
    - 4014 unique ids based on files
    - 2510 unique ids based on master
    - 41973 rows in master and based on files


Before changed to class method w separate call to flush
    - 3591 Unique user ids
    - 47952 rows in master.db

After
    - 4014 Unique user ids
    - 67940 rows in master.db
    



In [11]:
likelyNonEmpty = 4352 - 332 - 129
likelyNonEmpty

3891

In [ ]:
conn = sqlite3.connect(environment.MASTER_DB)
r1 = conn.execute("select user_id, count(word) from word_map_deux group by user_id ")
print( r1.fetchone())
conn.close()

In [ ]:
conn = sqlite3.connect(environment.MASTER_DB)
curs = conn.cursor()  # Connect a cursor
    
r1 = curs.execute("select * from word_map_deux where user_id = 1460971")
# for i in range(0, 100):
#     print( r1.fetchone())

curs.close()

In [ ]:
"""The above tells me that the problem is the whole store getting sent to the server"""

# Explore data

In [ ]:
def master_row_generator():
    conn = sqlite3.connect(environment.MASTER_DB)
    r = conn.execute("select * from word_map_deux")
    while True:
        yield r.fetchone()

def db_row_generator(filepath):
    conn = sqlite3.connect(filepath)
    r = conn.execute("select * from word_map_deux")
    while True:
        yield r.fetchone()


In [ ]:
gen = master_row_generator()

In [ ]:
for i in range(0, 220):
    print(next(gen))

In [ ]:
f = '/Users/adam/Desktop/TwitterDataAnalysisLogs/dbs/wordmapping1.db'
dgen = db_row_generator(f)
for i in range(0, 220):
    print(next(dgen))

In [ ]:
f = '/Users/adam/Desktop/TwitterDataAnalysisLogs/dbs/wordmapping1.db'
conn = sqlite3.connect(f)
r = conn.execute("select * from word_map_deux")
for i in range(0, 5):
    print(r.fetchone())


In [ ]:
f = '/Users/adam/Desktop/TwitterDataAnalysisLogs/dbs/wordmapping9.db'
conn = sqlite3.connect(f)
r = conn.execute("select * from word_map_deux")
for i in range(0, 5):
    print(r.fetchone())


In [ ]:

f = '/Users/adam/Desktop/TwitterDataAnalysisLogs/master.db'
conn = sqlite3.connect(f)
r = conn.execute("select count(word) from word_map_deux")
print(r.fetchone())
conn.close()

# Timing

In [ ]:
%cd twitteranalysis

from functools import wraps
import time

from Loggers.CsvLoggers import log_query, log_query_timestamp
import environment


def time_and_log(fn):
    """
    Decorator to time operation of method
    From High Performance Python, p.27
    """
    @wraps(fn)
    def measure_time(*args, **kwargs):
        t1 = time.time()
        result = fn(*args, **kwargs)
        t2 = time.time()
        t2 = time.time()
        elapsed = t2 - t1
        log_query( elapsed )
        log_query_timestamp()
#         print(("@timefn:%s took %s seconds" % (fn.__name__, elapsed)))
        return result
    return measure_time


In [ ]:

@time_and_log
def test():
    time.sleep(2)
    print('done')
    

In [ ]:
test()